# Assignment 2 Part A - Getting the data on 10 K filings in a dataframe (10 marks)

#### In this part of the assignment you are provided a list of mutual funds and the companies that were dropped from their portfolios.  We want to obtain the 10-K filings for these companies.  In the file (Dropped_Companies.csv), there is a column called "Name of Stock".  Load the file, extract a list of all the companies in that column, then find the 10-K links for all these companies.  Feel free to utilize Beautiful Soup and Pandas.

#### First, start with the importing of the relevant libraries:

In [1]:
import pandas as pd
import urllib

#### You will be populating the following lists:

In [2]:
weblink = []
filing_type = []
company_name = []
date = []

#### Upload the Dropped_Companies.csv file.

In [3]:
df = pd.read_csv('Dropped_Companies.csv')

#### The information on 10-K, 10-K/A, 10-K 405 of a company is in the the following link:

##### 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=' + Company+Name (a space between the two companies is replaced by a '+') + '&type=10-K&dateb=&owner=exclude&count=100'

#### So, your task will be to collect information on Company Name, Date of Filing, Filing Type and Link to the 'Document' of the filing type for each of the companies in the final dropped list

#### Hint: Make sure to collect all the filing links in every companies page. Number of filings available for each company varies, so make sure you take care of that

In [4]:
# generat a list of company links to the information on 10-k... 
company_link = []
for name in df["Name of Stock"]:
    name_list = name.split(" ")
    name = ('+').join(name_list)
    link = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company='+name+ '&type=10-K&dateb=&owner=exclude&count=100'
    company_link.append(link)

In [5]:
# Using BeautifulSoup to get Filling Data/Filling Weblink/Filling Date
import bs4 as bs
from bs4 import SoupStrainer
import codecs
import time

for link_number in range(0,len(company_link)):
    html = urllib.request.urlopen(company_link[link_number])
    html = html.read().decode('utf-8')
    
    soup = bs.BeautifulSoup(html)
    
    infotable = soup.find_all("table", summary = "Results")
    if infotable[0].find_all("tr")[0].find_all("th")[0].getText() == "Filings":
        for row in infotable[0].find_all("tr")[1:]:
            columns = row.find_all("td")
            date.append(columns[3].getText())
            filing_link = columns[1].find("a").get("href")
            weblink.append('https://www.sec.gov'+filing_link)
            type = filing_link[columns[1].find("a").get("href").find(".")+1:]
            filing_type.append(columns[0].getText())
            company_name.append(df["Name of Stock"][link_number])
            


#### After populating the lists, you need to save the data in the following csv file, using what you have learned from the pandas lesson

In [6]:
outputdf = pd.DataFrame({"Company Name":company_name,"Date of Filing":date, "Filing Type":filing_type,"Link":weblink})
outputdf.to_csv('10K Links.csv')